负责获取查询并返回相关文档

In [1]:
from langchain.document_loaders import TextLoader
# 定义文本加载器，指定读取的文件路径
loader = TextLoader("./documents/三国演义.txt")
documents=loader.load()

from langchain.text_splitter  import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512,chunk_overlap=20,    
                                      separators=[
                                                "\n\n",
                                                "\n",
                                                " ",
                                                ".",
                                                ",",
                                                "\u200b",  # Zero-width space
                                                "\uff0c",  # Fullwidth comma
                                                "\u3001",  # Ideographic comma
                                                "\uff0e",  # Fullwidth full stop
                                                "\u3002",  # Ideographic full stop
                                                "",
                                            ],)
split_documents=text_splitter.split_documents(documents)
print(len(split_documents))

25


In [2]:
# 加载embedding模型
from langchain.embeddings.ollama import OllamaEmbeddings

embeddings_model=OllamaEmbeddings(model='quentinz/bge-large-zh-v1.5:latest')

C:\Users\wushaogui\AppData\Local\Temp\ipykernel_25848\1405274082.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings_model=OllamaEmbeddings(model='quentinz/bge-large-zh-v1.5:latest')


In [3]:
from langchain.vectorstores import Chroma

persist_directory="VectorStore" # 数据库保存路径
db = Chroma.from_documents(split_documents, embeddings_model, persist_directory=persist_directory)
db.persist()

C:\Users\wushaogui\AppData\Local\Temp\ipykernel_25848\2049267115.py:5: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [5]:
# 实例化一个检索器
retriever = db.as_retriever()
# 使用检索器查询查询文本
docs = retriever.invoke("曹操如何逃出董卓的掌握？")
print(len(docs),docs[0:2])

4 [Document(metadata={'source': './documents/三国演义.txt'}, page_content='董卓这样残暴，大臣们愤恨不平，打算找机会除掉董卓。渤海太守袁绍给司徒王允写了封密信，让王允设法除掉董卓。王允便请几位大臣到家，商议再三也想不出办法，王允放声大哭，大臣们也都跟着哭起来。\n\u3000\u3000曹操看大家正在哭，却大笑起来。王允气冲冲地责问曹操为何发笑。曹操说他愿借王允的七星宝刀，拼死杀掉董卓。王允非常高兴，便把宝刀交给了曹操。\n\u3000\u3000第二天，曹操带着宝刀来到相府，董卓问曹操为什么来迟，曹操撒谎说马不好。董卓便叫吕布去为曹操选一匹良马。曹操见屋内只剩董卓一人，便要拔刀下手。但又怕董卓力大，自己一人杀不了，便犹豫起来。\n\u3000\u3000董卓身体肥胖，坐了一会儿，感到疲倦，就倒身朝里睡下。曹操忙拔刀去刺，不料董卓从镜里看见曹操在背后拔刀，猛的一个翻身，问曹操干什么，曹操非常惊慌，急忙跪下说他有口宝刀献给董卓 。\n\u3000\u3000曹操骗过董卓，恰好吕布已将选好的马牵到了，曹操便借口试试马，一出城，就纵马向东南谯郡飞奔去了。董卓得知，才明白曹操刚才是要刺杀他，便下令立即画出曹操的图像，发到各地，捉拿曹操。\n\u3000\u3000曹操逃到中牟县时，被守城的兵士认出捉住。县令陈宫见曹操是一个英雄，就弃官与曹操一同逃走了。'), Document(metadata={'source': './documents/三国演义.txt'}, page_content='何进问曹操为什么发笑。曹操说∶“要杀宦官，应该先除为首的宦官，这只要一个监狱官就够了，何必纷纷召来外兵呢？”何进一听，非常生气，喝斥曹操有私心。曹操退了出来，叹息着说∶“乱天下的，一定是何进！”\n\u3000\u3000朝廷派往西凉领兵的刺史董卓，手下有二十万大军，野心很大。\n\u3000\u3000这天，他接到何进的檄文，心中十分高兴，便令他的女婿李儒先派人向朝廷上了一道表章，说这次出兵是为了惩除宦官，制止逆乱。好让朝廷的大臣不对他产生怀疑。随后便起兵向京城洛一陽一进发。\n\u3000\u3000张让等听说外地的兵来到，便先下手，派五十名刀斧手埋伏在长乐宫嘉德门内，让何太后召何进进宫。何太后不知是计，就派人宣何进

向量数据库默认是“相似度查询”，根据问题的语义去数据库中搜索与之相关性最高的文档，这会存在一个问题，如果数据库存在很多相似文本，搜索出现较多重复文本，此时可以使用“最大边际相关性(MMR)”，它同时考量查询与文档的相关度，以及文档之间的相似度，解决返回文档冗余的问题，使得返回文本多样性更充足

In [6]:
# 指定检索类型：最大边际相关性搜索
retriever = db.as_retriever(search_type="mmr")
# 使用检索器查询查询文本
docs = retriever.invoke("刘关张第一次出场是在哪里？")
print(len(docs),docs[0:2])

4 [Document(metadata={'source': './documents/三国演义.txt'}, page_content='这时，一推车大汉进店饮酒。刘备留神一看，这人有九尺高，胸前长须飘飘，脸色好像红枣一样，长一双丹凤眼，两条卧蚕眉，相貌非常威武雄伟。刘备连忙起身，邀他过来同坐，并请问姓名。\n\u3000\u3000那人说∶“我姓关名羽，字云长，因乡里恶霸仗势欺人，我一怒杀了恶霸，逃到外乡避难已有五、六年了。”刘备、张飞听了都很敬佩，也将自己的志愿告诉了关羽。关羽听了也非常高兴。\n\u3000\u3000酒后他们一同来到张飞的庄上，只见庄后有一座桃园，园中桃花灿烂，景色很美。第二天，三人在园中焚香礼拜，宣誓结为异姓兄弟，三人按年岁认了兄弟，刘备做了大哥，关羽第二，张飞最小，做了弟弟。\n\u3000\u3000三人请来铁匠打造兵器。刘备打造了双股剑，关羽打了把八十二斤的青龙偃月刀，张飞造了一支丈八点钢矛，各人又造了一身铠甲。\n\u3000\u3000他们聚集乡中壮士五百多人，浩浩荡荡到涿郡去应募。\n\u3000\u3000三人在涿郡打败了黄巾军将领程远志。刘备听说他从前的老师中郎将卢植在广宗和张角作战，便领了本部人马到广宗助战。卢植令刘备三兄弟前往颍川帮助官军作战。刘备、关羽、张飞引军连夜奔赴颍川。'), Document(metadata={'source': './documents/三国演义.txt'}, page_content='孙坚进洛一陽一城救火时，从皇宫的井里打捞起一颗皇帝用的玉玺大印。袁绍得知这事后要孙坚将传国玉玺归还朝廷，孙坚撒谎说他没有见过玉玺，带领人马回江东去了。\n\u3000\u3000袁绍大怒，写了书信，叫人连夜赶往荆州送与刺史刘表，要他在路上截击孙坚。诸侯们四分五裂，纷纷拔寨离去。讨伐董卓的军队就这样散去了。刘备、关羽、张飞又回到了平原县。')]


还可以为向量数据库设置检索的参数，比如相似度阈值，topK

In [14]:
# 指定相似度
retriever = db.as_retriever(search_kwargs={"k": 2}
)
# 使用检索器查询查询文本
docs = retriever.invoke("王允如何用计董卓的？")
print(docs[0])

page_content='李儒劝董卓把貂蝉赐给吕布，董卓向貂蝉说出这个想法，貂蝉大吃一惊，哭着说她宁死也不愿离开董卓。一边说，一边抽出墙上挂着的宝剑，便要自杀。董卓连忙抱住劝她，再也不提将她赐给吕布的事了。
　　吕布得不到貂蝉，只是痛恨叹息。王允见吕布怀恨董卓老贼，便请吕布到家商量如何杀掉董卓。
　　计议已定，王允便差骑都尉李肃前往眉坞传诏书给董卓。李肃说∶“皇帝的病刚好，要将皇位让给太师。”董卓并没有怀疑，当天便同李肃回到京城长安。
　　王允等早已安排好了，董卓车驾一到，埋伏的武士挺著刀槍见董卓便刺，董卓穿著厚甲，没有被刺死。董卓大叫吕布救命，吕布应声从车后转出，一戟直刺董卓咽喉，顿时气绝了。
　　王允下令将董卓尸体扔在街头，让老百姓泄恨。侍中蔡邕因感念董卓对自己的好处，对董卓的死伤痛至极。王允要杀蔡邕，众官说蔡邕很有才气，请求留他继续写完汉史。王允没有听众官的意见，还是把蔡邕打入狱中害死了。' metadata={'source': './documents/三国演义.txt'}


# 使用大模型生成上下文的MultiQueryRetriever

基于距离的向量数据库检索将查询嵌入高维空间，并根据距离度量找到类似的嵌入文档。但是，检索可能会因查询措辞的细微变化，或如果嵌入没有很好地捕捉数据的语义而产生不同的结果。提示工程/调整有时用于手动解决这些问题，但这可能很繁琐

MultiQueryRetriever 通过使用 LLM 为给定的用户输入查询生成多个不同视角的查询来自动化提示调整过程。对于每个查询，它都会检索一组相关文档，并获取所有查询的唯一并集以获得更大的潜在相关文档集。通过对同一问题的生成多种视角，MultiQueryRetriever 可以缓解基于距离检索的一些限制，并获得更丰富的结果

In [15]:
from langchain.retrievers.multi_query import MultiQueryRetriever

from langchain_ollama import OllamaLLM

# 初始化Ollama LLM，注意需要后台开启ollama服务
model_name = "qwen2.5:latest"
llm  = OllamaLLM(model=model_name)

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=db.as_retriever(), llm=llm
)

In [16]:
# 使用检索器查询查询文本
docs = retriever_from_llm.invoke("曹操攻打黄巾军成功了吗？")
print(len(docs),docs)

4 [Document(metadata={'source': './documents/三国演义.txt'}, page_content='再说张梁、张宝在颍川连胜几阵，这天正在追赶官军，忽然被一队打着红旗的队伍拦住去路。为首一将姓曹名操，字孟德。张梁、张宝打不过这支队伍，领兵败走。\n\u3000\u3000刘备见黄巾军退走，便引军返回广宗。半途中，忽见一支军马押着囚车而来。上前一看，车中犯人竟是卢植，慌忙下马询问原因，才明白左丰因卢植未奉送金银，便在皇帝面前使坏。张飞一听大怒，拔刀要杀押送囚车的官兵救出卢植。刘备急忙拦住，说朝廷自有公论。\n\u3000\u3000三人便一齐回涿县去。正进间，见黄巾军把董卓领导的官军杀得大败。三人冲入阵中，救出了董卓。不料董卓一听三人并无官职，立刻把三人丢在外边，下马进帐去了。张飞顿时火冒三丈，便拔刀进帐要杀董卓，又被刘备劝住。三人于是领着人马，连夜去投朱隽。\n\u3000\u3000朱隽那时正在与黄巾军作战，便令刘备为先锋去攻打张宝。刘备一箭射中张宝左臂，关羽、张飞一齐出马助战，打败了张宝。朱隽便领大军去攻打宛城。\n\u3000\u3000这时，张角兄弟先后战死，黄巾军只剩下数万人屯住宛城一带。\n\u3000\u3000朱隽在刘备、关羽、张飞和吴郡人孙坚的帮助下，占了宛城，打败了黄巾军。朱隽回到京城，被封为车骑将军、河南尹。'), Document(metadata={'source': './documents/三国演义.txt'}, page_content='两军无粮再战，只好暂时停战。\n\u3000\u3000陶谦这年已六十三岁，忽然得了重病，他再次要求刘备接受徐州太守职位。刘备坚决不从。不久陶谦死了，部下把徐州的牌印交给了刘备，刘备无法再推，只好从命，担起了管理徐州的重任。\n\u3000\u3000曹操听说刘备得了徐州，不禁大怒，便要传令出兵攻打徐州。谋士荀见曹操动怒，建议他攻打颍川等地的黄巾军，夺取当地粮食。\n\u3000\u3000曹操依计攻下颍川、汝南等地，降了谯县人许褚，封为都尉。\n\u3000\u3000曹操又收复了兖州、濮一陽一。吕布走投无路，前来徐州投靠刘备。\n\u3000\u3000刘备为人宽厚，尽管关羽、张飞坚决反对，还是把小沛让给了吕布暂住。'), Document(metada

# 使用大模型压缩检索到的数据

向量存储检索器是使用向量存储检索文档的检索器。它是一个围绕向量存储类的轻量级包装器，使其符合检索器接口。它使用向量存储实现的搜索方法（例如相似度搜索和 MMR）来查询向量存储中的文本


In [17]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_ollama import OllamaLLM

compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [18]:
# 使用检索器查询查询文本
docs = compression_retriever.invoke("虎牢关发生了什么？")
print(len(docs),docs)

1 [Document(metadata={'source': './documents/三国演义.txt'}, page_content='董卓得知华雄被杀，便领大军与盟军大战于虎牢关。吕布勇不可挡，接连杀败八路诸侯。')]
